In [11]:
import healpy as hp
import pandas as pd
import numpy as np
import astropy
from astropy import units as u
from astropy.coordinates import SkyCoord
%matplotlib inline
from healpy_m import obj_in_pix
from tqdm import tqdm_notebook

In [12]:
pc0 = pd.read_csv('/home/rt2122/data/Tile6_1_5deg_0.csv')

In [13]:
pc0.head()

,id,objID,raStack,decStack,gPSFFlux,gPSFFluxErr,gKronFlux,gKronFluxErr,rPSFFlux,rPSFFluxErr,...,iKronFlux,iKronFluxErr,zPSFFlux,zPSFFluxErr,zKronFlux,zKronFluxErr,yPSFFlux,yPSFFluxErr,yKronFlux,yKronFluxErr
0,0,187792166702849712,216.670292,66.499140,1.281900e-06,4.447640e-07,-999.000000,-9.990000e+02,0.000004,5.802500e-07,...,0.000004,4.520370e-07,0.000006,9.828530e-07,0.000008,0.000002,7.760270e-06,0.000003,-999.00000,-999.000000
1,0,187792166510120457,216.650981,66.491425,2.927060e-06,4.368490e-07,0.000004,8.035710e-07,0.000001,5.780550e-07,...,-999.000000,-9.990000e+02,0.000003,9.966210e-07,-999.000000,-999.000000,2.012660e-06,0.000002,-999.00000,-999.000000
2,0,187792166628682728,216.662838,66.493315,8.349020e-05,5.894640e-07,0.000237,1.625190e-06,0.000148,7.903330e-07,...,0.000610,1.877780e-06,0.000241,1.258340e-06,0.000726,0.000004,2.550230e-04,0.000002,0.00073,0.000008
3,0,187792166559463846,216.655974,66.494247,5.957160e-07,4.306020e-07,-999.000000,-9.990000e+02,0.000003,5.786240e-07,...,0.000002,4.063660e-07,0.000002,9.674790e-07,-999.000000,-999.000000,-9.766080e-07,0.000002,-999.00000,-999.000000
4,0,187802166383557052,216.638363,66.505259,1.023320e-06,4.590230e-07,-999.000000,-9.990000e+02,0.000003,5.788770e-07,...,0.000004,5.598810e-07,0.000005,1.064530e-06,-999.000000,-999.000000,3.418760e-06,0.000002,-999.00000,-999.000000


In [14]:
def proc_big_table(df):
    print('Size of table before: ', df.shape[0])
    df.drop_duplicates(subset = ['raStack', 'decStack'], keep='first', inplace=True)
    df = df[df['raStack'] != -999]
    df = df[df['decStack'] != -999]
    df.index = np.arange(df.shape[0])
    print('Size of table after: ', df.shape[0])
    return df

In [19]:
from threading import Thread
class MyThread(Thread):
    def __init__(self, table, st, en):
        Thread.__init__(self)
        self.table = table
        self.st = st
        self.en = en
    
    def run(self):
        for i in range(st, en):
            ra = self.table.loc[i, 'raStack']
            dec = self.table.loc[i, 'decStack']
            npix = obj_in_pix(ra=ra, dec=dec, nside=2**11)
            self.table.loc[i, 'pix'] = npix

In [16]:
pc0 = proc_big_table(pc0)

Size of table before:  3440408
Size of table after:  3425830


In [17]:
pc0['pix'] = None

In [18]:
st_0, en_0 = 0, 100000

In [20]:
for i in range(st_0,  en_0 // 1000):
    st = i * 1000
    en = min(st + 1000, pc0.shape[0])
    thr = MyThread(pc0, st, en)
    thr.start()

In [21]:
pc0.loc[100000]

id                               0
objID           187292213477671681
raStack                    221.348
decStack                   66.0758
gPSFFlux               1.78562e-05
gPSFFluxErr            4.97333e-07
gKronFlux              2.58127e-05
gKronFluxErr           8.43367e-07
rPSFFlux               2.69458e-05
rPSFFluxErr            4.80136e-07
rKronFlux              4.99269e-05
rKronFluxErr           9.68336e-07
iPSFFlux               2.89338e-05
iPSFFluxErr            3.81714e-07
iKronFlux              6.53492e-05
iKronFluxErr           1.01057e-06
zPSFFlux                3.9801e-05
zPSFFluxErr            8.41858e-07
zKronFlux              7.35634e-05
zKronFluxErr           1.66553e-06
yPSFFlux               4.28719e-05
yPSFFluxErr            2.49639e-06
yKronFlux              7.05774e-05
yKronFluxErr            4.6483e-06
pix                           None
Name: 100000, dtype: object

In [ ]:
pc0.to_csv('~/data/pc0_fixed.csv')